# TASK A - Q1

We have decided to use one-to-many embedded document modelling for this assignment. The hotspot data is embedded into climate data. In embedded model there is a limit size for each document which is 16 MB. We calculated that with this given data the estimated document number for exceeding the limit is around 290,000 documents which means we need to have 290,000 reports in a day. we thought that it is unrealistic because in the given year the highest hotspot report in a day is 357.
Another reason we use embedded modelling is that we want efficient querying. We are able to access both of the documents in one call, which makes the query - when both documents are called - is faster

In [22]:
our_model_embedded = {'_id': '5ce7e9519343696b2835baca',
 'air_temperature_celcius': 20.0,
 'date': '6/03/2017',
 'hotspot': [{'confidence': 87.0,
              'date': '6/03/2017',
              'datetime': '2017-03-06T05:06:30',
              'location': [{'latitude': -34.3795, 'longitude': 141.6331}],
              'surface_temperature_celcius': 62.0},
             {'confidence': 85.0,
              'date': '6/03/2017',
              'datetime': '2017-03-06T05:06:20',
              'location': [{'latitude': -34.3735, 'longitude': 141.6604}],
              'surface_temperature_celcius': 59.0}],
 'max_wind_speed': 21.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 52.9,
 'station': '948701',
 'windspeed_knots': 17.0}

# Task B - Q1

In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [3]:
climate_his = pd.read_csv("climate_historic.csv")
hotspot_his = pd.read_csv("hotspot_historic.csv")

### Implementation (Embedded data model)

In [18]:
database={'db':[]}


for idx,date1 in enumerate(climate_his['date']):
    database['db'].append({'station': str(climate_his.iloc[idx,0]),
                           'date': climate_his.iloc[idx,1],
                           'air_temperature_celcius': float(climate_his.iloc[idx,2]),
                           'relative_humidity': climate_his.iloc[idx,3],
                           'windspeed_knots': climate_his.iloc[idx,4],
                           'max_wind_speed': climate_his.iloc[idx,5],
                           'precipitation': climate_his.iloc[idx,6],
                           'hotspot':[]})
    for i,date2 in enumerate(hotspot_his['date']):
        if date2==date1:
            database['db'][idx]['hotspot'].append({'location':[{'latitude':hotspot_his.iloc[i,0],
                                                                'longitude':hotspot_his.iloc[i,1]}],
                                                   'datetime':hotspot_his.iloc[i,2],
                                                   'confidence': float(hotspot_his.iloc[i,3]),
                                                   'date': hotspot_his.iloc[i,4],
                                                   'surface_temperature_celcius': float(hotspot_his.iloc[i,5],)})


In [20]:
len(database['db'])

366

In [21]:
# Method 1: connect on the default host and port
client = MongoClient () # method 1: connect on the default host and port

In [22]:
#creating database
db = client.fit5148_assignment_db

In [23]:
#creating collection
collection_embedded = db.collection_embedded

In [24]:
#drops all the documents
db.collection_embedded.drop()

In [25]:
result = db.collection_embedded.insert_many(database['db'])

# Task B - Q2

### Q2.a

In [13]:
result=db.collection_embedded.find({"date": '10/12/2017'})
for i in result:
    pprint(i)

{'_id': ObjectId('5ce804cf9343696b9ff6fe61'),
 'air_temperature_celcius': 17.0,
 'date': '10/12/2017',
 'hotspot': [{'confidence': 50.0,
              'date': '10/12/2017',
              'datetime': '2017-12-10T00:57:59',
              'location': [{'latitude': -37.986999999999995,
                            'longitude': 144.005}],
              'surface_temperature_celcius': 38.0},
             {'confidence': 67.0,
              'date': '10/12/2017',
              'datetime': '2017-12-10T00:56:57',
              'location': [{'latitude': -34.289,
                            'longitude': 141.71200000000002}],
              'surface_temperature_celcius': 54.0}],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.5,
 'station': '948702',
 'windspeed_knots': 7.3}


### Q2.b

In [12]:
for i in db.collection_embedded.aggregate([{"$unwind": "$hotspot"},{"$match":{"hotspot.surface_temperature_celcius": {"$lte": 100, "$gte":65}}},
                                          {"$project":{'hotspot.location':1,
                                    'hotspot.surface_temperature_celcius':1,
                                    'hotspot.confidence':1,"_id":0}}]):
    pprint(i['hotspot'])


{'confidence': 94.0,
 'location': [{'latitude': -37.2284, 'longitude': 147.9187}],
 'surface_temperature_celcius': 73.0}
{'confidence': 97.0,
 'location': [{'latitude': -37.6572, 'longitude': 142.0703}],
 'surface_temperature_celcius': 80.0}
{'confidence': 84.0,
 'location': [{'latitude': -37.0193, 'longitude': 148.1459}],
 'surface_temperature_celcius': 71.0}
{'confidence': 100.0,
 'location': [{'latitude': -37.4229, 'longitude': 147.02700000000002}],
 'surface_temperature_celcius': 99.0}
{'confidence': 80.0,
 'location': [{'latitude': -37.0055, 'longitude': 148.1582}],
 'surface_temperature_celcius': 68.0}
{'confidence': 85.0,
 'location': [{'latitude': -37.4128, 'longitude': 147.0242}],
 'surface_temperature_celcius': 98.0}
{'confidence': 90.0,
 'location': [{'latitude': -34.357, 'longitude': 141.5361}],
 'surface_temperature_celcius': 67.0}
{'confidence': 93.0,
 'location': [{'latitude': -34.3539, 'longitude': 141.5547}],
 'surface_temperature_celcius': 72.0}
{'confidence': 90.0,
 

{'confidence': 90.0,
 'location': [{'latitude': -36.3435, 'longitude': 141.5513}],
 'surface_temperature_celcius': 67.0}
{'confidence': 93.0,
 'location': [{'latitude': -36.6942, 'longitude': 143.8021}],
 'surface_temperature_celcius': 72.0}
{'confidence': 100.0,
 'location': [{'latitude': -36.3459, 'longitude': 141.5586}],
 'surface_temperature_celcius': 91.0}
{'confidence': 93.0,
 'location': [{'latitude': -36.4046, 'longitude': 141.1534}],
 'surface_temperature_celcius': 73.0}
{'confidence': 96.0,
 'location': [{'latitude': -36.0594, 'longitude': 141.5907}],
 'surface_temperature_celcius': 79.0}
{'confidence': 100.0,
 'location': [{'latitude': -36.0576, 'longitude': 141.6122}],
 'surface_temperature_celcius': 94.0}
{'confidence': 93.0,
 'location': [{'latitude': -36.388000000000005, 'longitude': 141.8309}],
 'surface_temperature_celcius': 73.0}
{'confidence': 90.0,
 'location': [{'latitude': -36.094, 'longitude': 145.7646}],
 'surface_temperature_celcius': 67.0}
{'confidence': 99.0,

### Q2.c

In [14]:
result=db.collection_embedded.find({"date": {"$in": ['15/12/2017', '16/12/2017']}},
                                     {'hotspot.surface_temperature_celcius':1,
                                      'air_temperature_celcius':1,
                                      'relative_humidity':1,
                                      'max_wind_speed':1,
                                      'date':1, '_id':0})
for i in result:
     pprint(i)

{'air_temperature_celcius': 18.0,
 'date': '15/12/2017',
 'hotspot': [{'surface_temperature_celcius': 42.0},
             {'surface_temperature_celcius': 36.0},
             {'surface_temperature_celcius': 38.0},
             {'surface_temperature_celcius': 40.0}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18.0,
 'date': '16/12/2017',
 'hotspot': [{'surface_temperature_celcius': 43.0},
             {'surface_temperature_celcius': 33.0},
             {'surface_temperature_celcius': 54.0},
             {'surface_temperature_celcius': 73.0},
             {'surface_temperature_celcius': 55.0},
             {'surface_temperature_celcius': 75.0},
             {'surface_temperature_celcius': 55.0},
             {'surface_temperature_celcius': 66.0},
             {'surface_temperature_celcius': 56.0},
             {'surface_temperature_celcius': 60.0},
             {'surface_temperature_celcius': 73.0},
             {'surface_temperature_celcius': 48.0},


### Q2.d

In [15]:
for i in db.collection_embedded.aggregate([{"$unwind": "$hotspot"},{"$match":{"hotspot.surface_temperature_celcius": {"$lte": 100, "$gte":80}}},
                                          {"$project":{'air_temperature_celcius':1,'hotspot.datetime':1,
                                    'hotspot.surface_temperature_celcius':1,
                                    'hotspot.confidence':1,"_id":0}}]):
    pprint(i)

{'air_temperature_celcius': 19.0,
 'hotspot': {'confidence': 97.0,
             'datetime': '2017-03-10T04:45:30',
             'surface_temperature_celcius': 80.0}}
{'air_temperature_celcius': 21.0,
 'hotspot': {'confidence': 100.0,
             'datetime': '2017-03-12T04:28:20',
             'surface_temperature_celcius': 99.0}}
{'air_temperature_celcius': 21.0,
 'hotspot': {'confidence': 85.0,
             'datetime': '2017-03-12T04:27:20',
             'surface_temperature_celcius': 98.0}}
{'air_temperature_celcius': 21.0,
 'hotspot': {'confidence': 97.0,
             'datetime': '2017-03-18T03:50:40',
             'surface_temperature_celcius': 80.0}}
{'air_temperature_celcius': 24.0,
 'hotspot': {'confidence': 98.0,
             'datetime': '2017-03-19T04:33:00',
             'surface_temperature_celcius': 83.0}}
{'air_temperature_celcius': 24.0,
 'hotspot': {'confidence': 100.0,
             'datetime': '2017-03-19T04:32:50',
             'surface_temperature_celcius': 91.0}}
{'

### Q2.e

In [16]:
for i in db.collection_embedded.aggregate([{"$unwind":"$hotspot"},{"$sort" : {'hotspot.surface_temperature_celcius':-1}},{"$limit" : 10}]):
    pprint(i)

{'_id': ObjectId('5ce7e9519343696b2835baf5'),
 'air_temperature_celcius': 15.0,
 'date': '18/04/2017',
 'hotspot': {'confidence': 100.0,
             'date': '18/04/2017',
             'datetime': '2017-04-18T04:52:00',
             'location': [{'latitude': -38.1665, 'longitude': 143.062}],
             'surface_temperature_celcius': 124.0},
 'max_wind_speed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'_id': ObjectId('5ce7e9519343696b2835bae7'),
 'air_temperature_celcius': 16.0,
 'date': '4/04/2017',
 'hotspot': {'confidence': 100.0,
             'date': '4/04/2017',
             'datetime': '2017-04-04T04:32:50',
             'location': [{'latitude': -36.343, 'longitude': 142.1986}],
             'surface_temperature_celcius': 123.0},
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'_id': ObjectId('5ce7e9519343696b2835bb02'),
 'air_tempe

### Q2.f

In [17]:
result=db.collection_embedded.find({})
for i in result:
    print("date: " + str(i["date"]) + "\nFires: " + str(len(i["hotspot"])) + "\n")

date: 31/12/2016
Fires: 0

date: 2/01/2017
Fires: 0

date: 3/01/2017
Fires: 0

date: 4/01/2017
Fires: 0

date: 5/01/2017
Fires: 0

date: 6/01/2017
Fires: 0

date: 7/01/2017
Fires: 0

date: 8/01/2017
Fires: 0

date: 9/01/2017
Fires: 0

date: 10/01/2017
Fires: 0

date: 11/01/2017
Fires: 0

date: 12/01/2017
Fires: 0

date: 13/01/2017
Fires: 0

date: 14/01/2017
Fires: 0

date: 15/01/2017
Fires: 0

date: 16/01/2017
Fires: 0

date: 17/01/2017
Fires: 0

date: 18/01/2017
Fires: 0

date: 19/01/2017
Fires: 0

date: 20/01/2017
Fires: 0

date: 21/01/2017
Fires: 0

date: 22/01/2017
Fires: 0

date: 23/01/2017
Fires: 0

date: 24/01/2017
Fires: 0

date: 25/01/2017
Fires: 0

date: 26/01/2017
Fires: 0

date: 27/01/2017
Fires: 0

date: 28/01/2017
Fires: 0

date: 29/01/2017
Fires: 0

date: 30/01/2017
Fires: 0

date: 31/01/2017
Fires: 0

date: 1/02/2017
Fires: 0

date: 2/02/2017
Fires: 0

date: 3/02/2017
Fires: 0

date: 4/02/2017
Fires: 0

date: 5/02/2017
Fires: 0

date: 6/02/2017
Fires: 0

date: 7/02/2017

### Q2.g

In [17]:
for i in db.collection_embedded.find({}):
    sum_surface = 0
    len_surface = len(i['hotspot'])
    avg_temp = 0
    for j in i['hotspot']:
        temp = j['surface_temperature_celcius']
        sum_surface += temp
    if sum_surface == 0:
        avg_temp = 'Nil'
    else:
        avg_temp = sum_surface/len_surface
    print("Date:" + str(i['date']) + "\nAvg Surface Temp: " + str(avg_temp) + "\n") 
        

Date:31/12/2016
Avg Surface Temp: Nil

Date:2/01/2017
Avg Surface Temp: Nil

Date:3/01/2017
Avg Surface Temp: Nil

Date:4/01/2017
Avg Surface Temp: Nil

Date:5/01/2017
Avg Surface Temp: Nil

Date:6/01/2017
Avg Surface Temp: Nil

Date:7/01/2017
Avg Surface Temp: Nil

Date:8/01/2017
Avg Surface Temp: Nil

Date:9/01/2017
Avg Surface Temp: Nil

Date:10/01/2017
Avg Surface Temp: Nil

Date:11/01/2017
Avg Surface Temp: Nil

Date:12/01/2017
Avg Surface Temp: Nil

Date:13/01/2017
Avg Surface Temp: Nil

Date:14/01/2017
Avg Surface Temp: Nil

Date:15/01/2017
Avg Surface Temp: Nil

Date:16/01/2017
Avg Surface Temp: Nil

Date:17/01/2017
Avg Surface Temp: Nil

Date:18/01/2017
Avg Surface Temp: Nil

Date:19/01/2017
Avg Surface Temp: Nil

Date:20/01/2017
Avg Surface Temp: Nil

Date:21/01/2017
Avg Surface Temp: Nil

Date:22/01/2017
Avg Surface Temp: Nil

Date:23/01/2017
Avg Surface Temp: Nil

Date:24/01/2017
Avg Surface Temp: Nil

Date:25/01/2017
Avg Surface Temp: Nil

Date:26/01/2017
Avg Surface Temp: